In [1]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [2]:
import pandas as pd

In [3]:
import os

# 学習データ、テストデータの読み込み
# kaggle環境で実行する場合
# train = pd.read_csv('../input/titanic/train.csv')
# test = pd.read_csv('../input/titanic/test.csv')
# ローカル環境で実行する場合
train = pd.read_csv('../../datasets/titanic/train.csv')
test = pd.read_csv('../../datasets/titanic/test.csv')

# 学習データを特徴量と目的変数に分ける
train_x = train.drop(['Survived'], axis=1)
train_y = train['Survived']

test_x = test.copy()


In [4]:
from sklearn.preprocessing import LabelEncoder

# 変数PassengerIdを除外する
train_x = train_x.drop(['PassengerId'], axis=1)
test_x = test_x.drop(['PassengerId'], axis=1)
test_x.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
# 変数Name, Ticket, Cabinを除外
train_x = train_x.drop(['Name', 'Ticket', 'Cabin'], axis=1)
test_x = test_x.drop(['Name', 'Ticket', 'Cabin'], axis=1)
test_x.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.5,0,0,7.8292,Q
1,3,female,47.0,1,0,7.0000,S
2,2,male,62.0,0,0,9.6875,Q
3,3,male,27.0,0,0,8.6625,S
4,3,female,22.0,1,1,12.2875,S


In [6]:
# それぞれのカテゴリ変数にLabel encodingを適用する
for c in ['Sex', 'Embarked']:
    # 学習データに基づいてどう変換するかを定める
    le = LabelEncoder()
    le.fit(train_x[c].fillna('NA'))

    # 学習データ、テストデータを変換する
    train_x[c] = le.transform(train_x[c].fillna('NA'))
    test_x[c] = le.transform(test_x[c].fillna('NA'))


In [7]:
test_x.head(100)
# csvに書き出し
test_x.to_csv('test_x.csv', index=False)


In [8]:
# 勾配ブースティングのライブラリをインポート
from xgboost import XGBClassifier

# モデルの作成および学習データを与えての学習
model = XGBClassifier(n_estimators=100, random_state=71)
model.fit(train_x, train_y)

# テストデータの予測値を確率で出力する
pred = model.predict_proba(test_x)[:, 1]


In [9]:
import numpy as np
# 予測値を2値に変換する
pred_label = np.where(pred > 0.5, 1, 0)

In [10]:
pred_label

array([0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [11]:
# 提出用のファイルの作成
submission = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': pred_label})
submission.to_csv('submission.csv', index=False)

In [12]:
train_x.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,22.0,1,0,7.2500,3
1,1,0,38.0,1,0,71.2833,0
2,3,0,26.0,0,0,7.9250,3
3,1,0,35.0,1,0,53.1000,3
4,3,1,35.0,0,0,8.0500,3


# クロスバリデーション

In [ ]:
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import KFold

# 各foldのスコアを保存するリスト
scores_accuracy = []
scores_logloss = []

# クロスバリデーションを行う
# 学習データを4つに分割し、うち一つをバリデーションデータとすることを、バリデーションデータを変えて繰り返す
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for tr_idx, va_idx, in kf.split(train_x):
    # 学習データを学習データとバリデーションデータに分ける
    # tr_idx, va_idxは分割後のindex
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
    
    # モデルの学習を行う
    model = XGBClassifier(n_estimators = 20, random_state=71)
    model.fit(tr_x, tr_y)
    
    # バリデーションデータの予測値を確率で出力する
    va_pred = model.predict_proba(va_x)[:,1]

    # バリデーションデータのスコアを計算する
    logloss = log_loss(va_y, va_pred)
    accuracy = accuracy_score(va_y, va_pred > 0.5)

    # 結果をリストに追加
    scores_accuracy.append(accuracy)
    scores_logloss.append(logloss)

# 各foldのスコアの平均を出力する　
mean_of_logloss = np.mean(scores_logloss)
mean_of_acc = np.mean(scores_accuracy)
print(f'logloss: {mean_of_logloss:.4f}, accuracy: {mean_of_acc:.4f}')
print(scores_accuracy)


logloss: 0.4405, accuracy: 0.8227
[0.8161434977578476, 0.8116591928251121, 0.8295964125560538, 0.8333333333333334]


# モデルのチューニング

In [20]:
import itertools

# チューニング候補とするパラメータを準備する
param_space = {
    'max_depth': [3, 5, 7],
    'min_child_weight': [1.0, 2.0, 4.0],
    'learning_rate': [0.01, 0.1, 0.3]
}

# 探索するハイパーパラメータの組み合わせ, productはデカルト積を返す
param_combinations = list(itertools.product(param_space['max_depth'], param_space['min_child_weight'], param_space['learning_rate']))

# 各パラメータの組み合わせ、それに対するスコアを保持するリスト
params = []
scores = []
# 各パラメータの組み合わせごとにクロスバリデーションで評価を行う
for max_depth, min_child_weight, learning_rate in param_combinations:
    scores_folds = []

    kf = KFold(n_splits=4, shuffle=True, random_state=71)
    for tr_idx, va_idx, in kf.split(train_x):
        # 学習データを学習データとバリデーションデータに分ける
        # tr_idx, va_idxは分割後のindex
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        
        # モデルの学習を行う
        model = XGBClassifier(n_estimators = 20, random_state=71, 
                              max_depth=max_depth, min_child_weight=min_child_weight, learning_rate=learning_rate)
        model.fit(tr_x, tr_y)
        
        # バリデーションデータの予測値を確率で出力する
        va_pred = model.predict_proba(va_x)[:,1]
        logloss = log_loss(va_y, va_pred)
        scores_folds.append(logloss)
    
    # 各foldのスコアを平均する　
    score_mean = np.mean(scores_folds)
    
    # パラメータの組み合わせ、それに対するスコアを保持する
    params.append((max_depth, min_child_weight, learning_rate))
    scores.append(score_mean)

# 最もスコアが良いものをベストパラメータとする
best_idx = np.argsort(scores)[0]
best_param = params[best_idx]
print(f'max_depth: {best_param[0]}, min_child_weight: {best_param[1]}, learning_rate: {best_param[2]}')

max_depth: 3, min_child_weight: 2.0, learning_rate: 0.3


In [21]:
best_param

(3, 2.0, 0.3)

# アンサンブル

In [ ]:
# ロジスティック回帰モデル用のデータを用意
from sklearn.preprocessing import OneHotEncoder

# 元データをコピーする
train_x2 = train.drop(['Survived'], axis=1)
test_x2 = test.copy()

# 変数PassengerIdを除外する
train_x2 = train_x2.drop(['PassengerId'], axis=1)
test_x2= test_x2.drop(['PassengerId'], axis=1)

# 変数Name, Ticket, Cabinを除外する
train_x2 = train_x2.drop(['Name', 'Ticket', 'Cabin'], axis=1)
test_x2 = test_x2.drop(['Name', 'Ticket', 'Cabin'], axis=1)

# one-hot encodingを行う
cat_cols = ['Sex', 'Embarked', 'Pclass']
ohe = OneHotEncoder(categories='auto', sparse=False)
ohe.fit(train_x2[cat_cols].fillna('NA'))





/Users/shosei/anaconda3/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


OneHotEncoder(sparse=False, sparse_output=False)

In [27]:
# one-hot encodingのダミー変数の列名を作成する
ohe_columns = []
for i, c in enumerate(cat_cols):
    ohe_columns += [f'{c}_{v}' for v in ohe.categories_[i]]
ohe_columns

['Sex_female',
 'Sex_male',
 'Embarked_C',
 'Embarked_NA',
 'Embarked_Q',
 'Embarked_S',
 'Pclass_1',
 'Pclass_2',
 'Pclass_3']

In [28]:
# one-hot encodingによる変換を行う
ohe_train_x2 = pd.DataFrame(ohe.transform(train_x2[cat_cols].fillna('NA')), columns=ohe_columns)
ohe_test_x2 = pd.DataFrame(ohe.transform(test_x2[cat_cols].fillna('NA')), columns=ohe_columns)


# one-hot encoding済みの変数を除外する
train_x2 = train_x2.drop(cat_cols, axis=1)
test_x2 = test_x2.drop(cat_cols, axis=1)

# 結合する
train_x2 = pd.concat([train_x2, ohe_train_x2], axis=1)
test_x2 = pd.concat([test_x2, ohe_test_x2], axis=1)

train_x2.head()

KeyError: "None of [Index(['Sex', 'Embarked', 'Pclass'], dtype='object')] are in the [columns]"

In [ ]:
from sklearn.linear_model import LogisticRegression

# xgboost
model_xgb = XGBClassifier(n_estimators=20, random_state=71, 
                          max_depth=best_param[0], min_child_weight=best_param[1], learning_rate=best_param[2])
model_xgb.fit(train_x, train_y)
pred_xgb = model_xgb.predict_proba(test_x)[:,1]

# ロジスティック回帰モデル
